You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv() # Загружает переменные из .env файла в окружение

# Получение ключа из переменной окружения
api_key = os.getenv('OPENAI_API_KEY')

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano'
os.environ["OPENAI_API_BASE"] = 'https://api.proxyapi.ru/openai/v1'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
        "6. Translate all the resulting text to Russian "
            "so that it looked like written by native Russian speaker.\n"
    ),
    expected_output="A well-written blog post in Russian "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice. Make sure it is in Russian language only."),
    expected_output="A well-written blog post in Russian, in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Application of LLM to optimize taxi services"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b7337795-7974-4ccf-91b4-a26712e0a6cb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: be7b8183-0970-4b03-bcf8-27ee2f6b0e34
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: be7b8183-0970-4b03-bcf8-27ee2f6b0e34
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Application of LLM to optimize taxi services.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


🤖 Agent: Content Planner
    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice. Make sure it is in Russian language only.


# Agent: Editor
## Final Answer: 
# Применение больших языковых моделей (LLM) для оптимизации такси-услуг

## Вступление: революция в сфере такси благодаря искусственному интеллекту

За последние годы индустрия такси претерпела значительные изменения благодаря цифровизации и внедрению передовых технологий. Современные компании стремятся обеспечить быстрое и удобное обслуживание, а также повысить эффективность своих операций, снизить издержки и улучшить взаимодействие с клиентами. В этом контексте большие языковые модели (LLM) — такие как GPT-4 и их аналоги — становятся мощным инструментом комплексной трансформации таксопарков и сервисов.

Обладая способностью анализировать огромные объемы данных и генерировать осмысленные ответы, LLM позволяют реализовать новые уровни автоматизации и персонализации. В этой статье мы рассмотрим, 

🚀 Crew: crew
├── 📋 Task: be7b8183-0970-4b03-bcf8-27ee2f6b0e34
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 516fb30a-8fe4-4667-82b4-aaf93dff1ec2
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: e9ecc876-623f-43f6-a885-1855d444b24e
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: be7b8183-0970-4b03-bcf8-27ee2f6b0e34
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 516fb30a-8fe4-4667-82b4-aaf93dff1ec2
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: e9ecc876-623f-43f6-a885-1855d444b24e
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e9ecc876-623f-43f6-a885-1855d444b24e                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b7337795-7974-4ccf-91b4-a26712e0a6cb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

# Применение больших языковых моделей (LLM) для оптимизации такси-услуг

## Вступление: революция в сфере такси благодаря искусственному интеллекту

За последние годы индустрия такси претерпела значительные изменения благодаря цифровизации и внедрению передовых технологий. Современные компании стремятся обеспечить быстрое и удобное обслуживание, а также повысить эффективность своих операций, снизить издержки и улучшить взаимодействие с клиентами. В этом контексте большие языковые модели (LLM) — такие как GPT-4 и их аналоги — становятся мощным инструментом комплексной трансформации таксопарков и сервисов.

Обладая способностью анализировать огромные объемы данных и генерировать осмысленные ответы, LLM позволяют реализовать новые уровни автоматизации и персонализации. В этой статье мы рассмотрим, как именно эти технологии помогают оптимизировать диспетчеризацию, повысить качество обслуживания и обеспечить безопасность водителей и пассажиров. В целом, применение LLM обещает вывести индустрию такси на новый уровень эффективности и инноваций.

## Роль LLM в операционной деятельности такси-сервисов

Одним из ключевых преимуществ LLM является автоматизация клиентского сервиса. Интеллектуальные чатботы, основанные на больших языковых моделях, способны круглосуточно отвечать на вопросы пассажиров, принимать заказы и разрешать споры без необходимости вмешательства оператора. Это значительно сокращает время реакции и снижает нагрузку на колл-центры.

Кроме того, LLM активно используются в области диспетчеризации и маршрутизации. Используя предиктивное моделирование, они помогают предугадывать пики спроса и оптимально распределять автомобили по районам. В результате снижается время ожидания и увеличивается количество выполненных заказов. Также модели анализируют данные о трафике и погодных условиях для динамического регулирования цен и маршрутов, делая сервис более гибким и конкурентоспособным.

## Улучшение клиентского опыта с помощью LLM

Персонализация — одна из сильных сторон LLM. Модели способны запоминать предпочтения пассажиров, такие как любимые маршруты или музыкальные настройки, и предлагать их при последующих заказах. Это повышает уровень удовлетворенности и укрепляет лояльность клиентов.

Более того, модели обеспечивают многоязычную поддержку, что облегчает работу в глобальных масштабах. Клиенты могут общаться с системой на своих языках без труда. Также автоматизированные системы обработки обратной связи и разрешения споров помогают быстро реагировать на претензии, повышая качество сервиса. В совокупности, использование таких технологий делает взаимодействие с клиентом максимально удобным и интеллектуальным.

## Безопасность и поддержка водителей

Большие языковые модели находят широкое применение и в сфере безопасности. Они могут генерировать протоколы реагирования при авариях и других инцидентах, своевременно уведомлять водителей о потенциальных угрозах и предоставлять навигационные советы. Анализируя данные о происшествиях, LLM помогают выявлять опасные ситуации и внедрять меры профилактики.

Кроме того, системы поддержки на базе LLM предоставляют водителям актуальную информацию о пробках, дорожных работах и новых правилах дорожного движения, что повышает безопасность и эффективность работы. В будущем эти технологии могут сыграть важнейшую роль в становлении полностью автономных такси, снижая риски и повышая доверие к автоматизированным системам перевозок.

## Барьеры и этические аспекты внедрения LLM

Несмотря на очевидные преимущества, использование больших языковых моделей в такси-отрасли сталкивается с рядом вызовов. В первую очередь, актуальны вопросы конфиденциальности данных пассажиров и соблюдения законодательства о защите личной информации. Обеспечение высокого уровня безопасности и прозрачности обработки данных является приоритетом.

Также существует риск моделирования ошибок или предвзятости, что может негативно сказаться на качестве обслуживания или привести к дискриминации отдельных групп пользователей. Интеграция новых решений в существующие системы требует времени и инвестиций, а нормативно-правовая база всё еще развивается. Поэтому внедрение LLM требует аккуратного подхода, с учетом этических и технологических аспектов.

## Перспективы и будущее развитие технологий LLM в сфере такси

Будущее применения больших языковых моделей в транспортной индустрии выглядит многообещающим. В рамках интеграции мультимодальных систем предполагается объединение данных из различных источников: карт, камер, датчиков — для создания более точных моделей спроса и повышения безопасности.

Также растет интерес к использованию LLM в городском планировании и технологиях дополненной реальности для автономных автомобилей, что может кардинально изменить облик городской мобильности. Следующим этапом станут масштабные внедрения AI-решений в полностью автономные такси, что откроет новую эру транспортных услуг.

## Итог: инновации, изменяющие индустрию

Подводя итог, можно сказать, что применение больших языковых моделей революционно меняет работу таксомоторных компаний, делая услуги более интеллектуальными, безопасными и персонализированными. Интеграция LLM способствует повышению операционной эффективности, снижению издержек и улучшению клиентского опыта.

Несмотря на существующие вызовы, перспективы развития технологий впечатляют и требуют активного внедрения и тестирования. Для бизнеса важно своевременно адаптироваться к этим изменениям, инвестировать в инновационные решения и использовать потенциал LLM для достижения конкурентных преимуществ.

Если вы заинтересованы в изучении передовых решений или хотите запустить пилотные проекты по внедрению искусственного интеллекта, рекомендуем следить за актуальными новостями и обращаться к экспертам в области AI и транспортных технологий.

---

Будущее таксомоторной индустрии — за технологическими инновациями, и большие языковые модели уже сегодня делают этот путь более умным, безопасным и комфортным для всех участников процесса.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Выбор между Kotlin и Python для разработки систем ИИ"
result = crew.kickoff(inputs={"topic": topic})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b7337795-7974-4ccf-91b4-a26712e0a6cb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: be7b8183-0970-4b03-bcf8-27ee2f6b0e34
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: be7b8183-0970-4b03-bcf8-27ee2f6b0e34
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Выбор между Kotlin и Python для разработки систем ИИ.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Comprehensive Content Plan for "Выбор между Kotlin и Python для разработки систем ИИ"**

---

### 1. Prioritized Latest Trends, Key Players, and Noteworthy News

**Latest Trends:**
- **Python dominance in AI development:** Python remains the most popular language for AI and machine learning, thanks to extensive libraries like TensorFlow, Keras, PyTorch, and scikit-learn.
- **Kotlin's rising role in AI:** While initially focused on Android development, Kotlin is gaining attention for backend and cross-platform development, with some emerging use cases in AI, e

🚀 Crew: crew
└── 📋 Task: be7b8183-0970-4b03-bcf8-27ee2f6b0e34
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: ✅ Completed

...

# Agent: Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice. Make sure it is in Russian language only.


# Agent: Editor
## Final Answer: 
Your final answer must be the great and the most complete as possible, it must be outcome described.




🚀 Crew: crew
├── 📋 Task: be7b8183-0970-4b03-bcf8-27ee2f6b0e34
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 516fb30a-8fe4-4667-82b4-aaf93dff1ec2
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: e9ecc876-623f-43f6-a885-1855d444b24e
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: be7b8183-0970-4b03-bcf8-27ee2f6b0e34
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 516fb30a-8fe4-4667-82b4-aaf93dff1ec2
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: e9ecc876-623f-43f6-a885-1855d444b24e
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e9ecc876-623f-43f6-a885-1855d444b24e                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b7337795-7974-4ccf-91b4-a26712e0a6cb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [15]:
Markdown(result.raw)

Your final answer must be the great and the most complete as possible, it must be outcome described.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).